# Reminder 
1. download the yelp dataset from https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset (We cannot upload the dataset to our github repo because even after zipping, it is over 4gb)

2. Unzip the downloaded file and put the files into "/src/backend-jupyter" directory (i.e. same directory with this .ipny file)

3. Press the "Run All" button under the cell column 
   (we didnt use colab to run the code because it is more convenient to process the data in local)
  
4. The output .json and .csv file will be export to the same directory


*After processing the data with this python notebook, we also manualy change some of the data in the dataset, i.e. change the data type from string to array by removing the ''

The following changes have been done manually:
In parallelCoordinate.json:
- change "stars" from string to int
- change 'Ambience' and 'categories' from string to array

In ranking.json
- change "value" from string to array

In wordCloudData.json
- change "categories" from string to array


In [38]:

import pandas as pd
import datetime
import csv 
import json 
import numpy as np

def csv_to_json(csvFilePath, jsonFilePath):
    jsonArray = []
      
    #read csv file
    with open(csvFilePath, encoding='utf-8') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf) 

        #convert each csv row into python dict
        for row in csvReader: 
            #add this python dict to json array
            jsonArray.append(row)
  
    #convert python jsonArray to JSON String and write to file
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonString = json.dumps(jsonArray, indent=4)
        jsonf.write(jsonString)
        
          

In [39]:
business_json_path = './yelp_academic_dataset_business.json'
df_b = pd.read_json(business_json_path, lines=True)

#drop closed restaurant 
df_b = df_b[df_b['is_open']==1]

#remove incomplete data
df_b = df_b.dropna()

#drop reduandant columns
drop_columns = ['address', 'postal_code', 'is_open']
df_b = df_b.drop(drop_columns, axis=1)

#drop duplicates
df_b = df_b.drop_duplicates(['name', 'latitude', 'longitude'])

business_R = df_b[df_b['categories'].str.contains(
              'Restaurants',
              case=False, na=False)]

In [40]:
duration=[]
#calculate opening duration of each restaurants
for day in business_R.hours:
    sum = datetime.timedelta(hours=0, minutes=0)
    for value in day.values():
        (start, end) = value.split('-')
        (h1, m1) = start.split(':')
        (h2, m2) = end.split(':')
        d1 = datetime.timedelta(hours=int(h1), minutes=int(m1))
        d2 = datetime.timedelta(hours=int(h2), minutes=int(m2))
        if (int(h2) > int(h1)):
            sum += (d2 - d1)
        elif (int(h2) == int(h1)):
            sum += datetime.timedelta(hours=24)
        else:
            sum += (d2 + datetime.timedelta(hours=24) - d1)
        total = sum.days*24 + sum.seconds/3600
        
    duration.append(total)
business_R.insert(len(business_R.columns), 'opening_hours', duration)
business_R = business_R.drop('hours', axis=1)
business_R.opening_hours



3          94.0
5         114.0
9         100.0
11         23.0
12        112.0
          ...  
150323     93.5
150325    168.0
150327    161.0
150336     74.0
150339    168.0
Name: opening_hours, Length: 31355, dtype: float64

In [41]:
#extract Ambience
Ambience = []
A_list = ["touristy: True", "hipster: True", "romantic: True", "divey: True", "intimate: True", "trendy: True", "upscale: True", "classy: True", "casual: True"]
for i in business_R.attributes:
    ans = []
    if ('Ambience' in i):
        temp = i.get('Ambience')
        temp = temp.replace("{", "[")
        temp = temp.replace("}", "]")
        temp = temp.replace('\"', '\"')
        temp = temp.replace('\'', '')
        temp = json.dumps(temp)
        
        for word in A_list:
            if word in temp:
                ans.append(word)
                
        if ans == []:
            Ambience.append('None')
        else:
            Result = ans
            Ambience.append(Result)
            
    else:
        Result = 'None'
        Ambience.append(Result)
    

#print(Ambience)
business_R.insert(len(business_R.columns), 'Ambience', Ambience)#
business_R.drop('attributes', axis=1)




cat = []

for i in business_R.categories:
    i = i.split(',')
    cat.append(i)


business_R = business_R.drop('categories', axis=1)
business_R.insert(len(business_R.columns), 'categories', cat)


print(business_R[:10])


    

               business_id                     name          city state  \
3   MTSW4McQd7CbVtyjqoe9mw       St Honore Pastries  Philadelphia    PA   
5   CF33F8-E6oudUQ46HnavjQ           Sonic Drive-In  Ashland City    TN   
9   bBDDEgkFA1Otx9Lfe7BZUQ           Sonic Drive-In     Nashville    TN   
11  eEOYSgkmpB90uNA7lDOMRA    Vietnamese Food Truck     Tampa Bay    FL   
12  il_Ro8jwPlHresjw9EGmBg                  Denny's  Indianapolis    IN   
15  MUTTqe8uqyMdBl186RmNeA                 Tuna Bar  Philadelphia    PA   
19  ROeacJQwBeh05Rqg7F6TCg                      BAP  Philadelphia    PA   
22  kfNv-JZpuN6TVNSO6hHdkw          Hibachi Express  Indianapolis    IN   
23  9OG5YkX1g2GReZM0AskizA  Romano's Macaroni Grill          Reno    NV   
29  sqSqqLy0sN8n2IZrAbzidQ           Domino's Pizza   White House    TN   

     latitude   longitude  stars  review_count  \
3   39.955505  -75.155564    4.0            80   
5   36.269593  -87.058943    2.0             6   
9   36.208102  -86.76817

In [5]:
df_explode = df_b.assign(categories = df_b.categories
                         .str.split(', ')).explode('categories')

In [6]:
df_explode.categories.value_counts()


Restaurants          31355
Food                 18045
Shopping             17731
Home Services        10970
Beauty & Spas        10420
                     ...  
Pathologists             1
Outdoor Movies           1
General Festivals        1
Osteopaths               1
Bubble Soccer            1
Name: categories, Length: 1278, dtype: int64

In [7]:
df_explode[df_explode.categories.str.contains('Restaurants',
                      case=True,na=False)].categories.value_counts()

Restaurants           31355
Pop-Up Restaurants       41
Name: categories, dtype: int64

In [8]:
review_json_path = './yelp_academic_dataset_review.json'

In [9]:
size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

In [10]:
chunk_list = []
for chunk_review in review:
    #Remove missing
    chunk_review.dropna()
    #Drop duplicates
    chunk_review.drop_duplicates(['review_id', 'text'])
    #Drop columns that aren't needed
    chunk_review = chunk_review.drop(['user_id', 'review_id','useful','funny','cool', 'text'], axis=1)
    #Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    #Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(business_R, chunk_review, on='business_id', how='inner')
    #Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
drop_columns = ['business_id']
df = df.drop(drop_columns, axis = 1)
df[:10]

529552 out of 1,000,000 related reviews
539114 out of 1,000,000 related reviews
513306 out of 1,000,000 related reviews
518310 out of 1,000,000 related reviews
533400 out of 1,000,000 related reviews
531474 out of 1,000,000 related reviews
527868 out of 1,000,000 related reviews


,name,city,state,latitude,longitude,stars,review_count,attributes,opening_hours,Ambience,categories,review_stars,date
0,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",94.0,None,"[Restaurants, Food, Bubble Tea, Coffee & Te...",4,2014-05-26 01:09:53
1,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",94.0,None,"[Restaurants, Food, Bubble Tea, Coffee & Te...",4,2013-10-05 15:19:06
2,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",94.0,None,"[Restaurants, Food, Bubble Tea, Coffee & Te...",5,2013-10-25 01:34:57
3,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",94.0,None,"[Restaurants, Food, Bubble Tea, Coffee & Te...",5,2018-05-20 17:58:57
4,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",94.0,None,"[Restaurants, Food, Bubble Tea, Coffee & Te...",4,2013-10-25 02:31:35
5,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",94.0,None,"[Restaurants, Food, Bubble Tea, Coffee & Te...",4,2018-03-17 23:47:25
6,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",94.0,None,"[Restaurants, Food, Bubble Tea, Coffee & Te...",3,2012-08-16 19:42:17
7,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",94.0,None,"[Restaurants, Food, Bubble Tea, Coffee & Te...",4,2017-04-20 12:37:09
8,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",94.0,None,"[Restaurants, Food, Bubble Tea, Coffee & Te...",4,2014-02-25 14:41:08
9,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",94.0,None,"[Restaurants, Food, Bubble Tea, Coffee & Te...",4,2012-01-10 02:09:18


In [11]:
##heatMapData.json
drop_columns = ['name', 'city', 'state', 'latitude', 'longitude', 'stars', 'review_count', 'categories', 'opening_hours', 'Ambience', 'review_stars', 'attributes']
heatMap = df.drop(drop_columns, axis = 1)



key = []
count = 0
for i in heatMap[heatMap.columns[0]]:
    key.append(count)
    count = count + 1
heatMap.insert(len(heatMap.columns), 'key', key)

print(heatMap[:10])

csv_name = "./heatMapData.csv"
heatMap.to_csv(csv_name, index=False)

          
csvFilePath = r'./heatMapData.csv'
jsonFilePath = r'./heatMapData.json'
csv_to_json(csvFilePath, jsonFilePath)


                  date  key
0  2014-05-26 01:09:53    0
1  2013-10-05 15:19:06    1
2  2013-10-25 01:34:57    2
3  2018-05-20 17:58:57    3
4  2013-10-25 02:31:35    4
5  2018-03-17 23:47:25    5
6  2012-08-16 19:42:17    6
7  2017-04-20 12:37:09    7
8  2014-02-25 14:41:08    8
9  2012-01-10 02:09:18    9


In [12]:
review_jsonO_path = './Obesity_2021.json'
  
data = pd.read_json(review_jsonO_path)
data

,state,state_Full,prevalence
0,AL,Alabama,39.9
1,AK,Alaska,33.5
2,AZ,Arizona,31.3
3,AR,Arkansas,38.7
4,CA,California,27.6
5,CO,Colorado,25.1
6,CT,Connecticut,30.4
7,DE,Delaware,33.9
8,DC,District of Columbia,24.7
9,FL,Florida,No data


In [13]:
#data = pd.merge(data, business_R, on='state', how='inner')

#data[:10]

In [14]:
###ranking.json
ranking = pd.merge(data, df, on='state', how='inner')


drop_columns = ['state', 'prevalence', 'city', 'latitude', 'longitude', 'stars', 'review_count', 'attributes', 'opening_hours', 'opening_hours', 'Ambience', 'categories', 'date']
ranking = ranking.drop(drop_columns, axis = 1)

ranking['stars'] = ranking.groupby('name')['review_stars'].transform('mean')
ranking = ranking.drop_duplicates(['name'])
ranking = ranking.drop(['review_stars'], axis = 1)

ranking = ranking.rename(columns={'stars':'value', 'name': 'text', 'review_stars': 'value'})
ranking.value = pd.to_numeric(ranking.value)

ranking[:10]
csv_name = "./ranking.csv"
ranking.to_csv(csv_name, index=False)


          
csvFilePath = r'./ranking.csv'
jsonFilePath = r'./ranking.json'
csv_to_json(csvFilePath, jsonFilePath)






In [50]:
###parallelCoordinate.json

parallelCoordinate = pd.merge(data, business_R, on='state', how='inner')

drop_columns = ['state', 'latitude', 'longitude', 'review_count', 'city', 'name', 'business_id', 'prevalence', 'attributes']
parallelCoordinate = parallelCoordinate.drop(drop_columns, axis = 1)
parallelCoordinate[:10]


,state_Full,stars,opening_hours,Ambience,categories
0,Arizona,3.5,112.0,[casual: True],"[Bars, Restaurants, Sports Bars, Nightlife]"
1,Arizona,4.0,103.0,"[trendy: True, casual: True]","[Sports Bars, Bars, Nightlife, Italian, Re..."
2,Arizona,4.0,30.0,"[classy: True, casual: True]","[Restaurants, Salad, Pakistani, Indian, Co..."
3,Arizona,4.0,42.5,[casual: True],"[Restaurants, Chinese]"
4,Arizona,5.0,36.0,None,"[Food Trucks, Restaurants, Caterers, Event ..."
5,Arizona,3.0,76.0,None,"[Event Planning & Services, Caterers, Fast F..."
6,Arizona,3.5,112.0,"[divey: True, casual: True]","[Nightlife, American (New), Music Venues, R..."
7,Arizona,2.5,93.0,None,"[Fast Food, Burgers, Restaurants]"
8,Arizona,4.5,36.0,"[hipster: True, classy: True, casual: True]","[Event Planning & Services, Food, Sandwiches..."
9,Arizona,2.5,96.5,None,"[Food Delivery Services, Restaurants, Pizza,..."


In [51]:


csv_name = "./parallelCoordinate.csv"
parallelCoordinate.to_csv(csv_name, index=False)



csvFilePath = r'./parallelCoordinate.csv'
jsonFilePath = r'./parallelCoordinate.json'
csv_to_json(csvFilePath, jsonFilePath)


In [17]:
####mapData.json
#read business drop column, merge with prevalence, drop prevalence


business_json_path = './yelp_academic_dataset_business.json'
df_b = pd.read_json(business_json_path, lines=True)

#drop closed restaurant 
df_b = df_b[df_b['is_open']==1]

#remove incomplete data
#key_cols = ['business_id', 'name', 'city', 'state', 'latitude', 'longitude', 'stars', 'categories', 'stars']
#df_b = df_b.dropna(subset = key_cols)
df_b = df_b.dropna()
#print('Percentage of missing values are' + format((1 - df_b1.count() / df_b.count()) * 100))

#drop reduandant columns
drop_columns = ['business_id','address', 'city', 'postal_code', 'is_open', 'attributes', 'hours']
df_b = df_b.drop(drop_columns, axis=1)

#drop duplicates
df_b = df_b.drop_duplicates(['name', 'latitude', 'longitude'])

business_R = df_b[df_b['categories'].str.contains(
              'Restaurants',
              case=False, na=False)]
business_R[:10]

,name,state,latitude,longitude,stars,review_count,categories
3,St Honore Pastries,PA,39.955505,-75.155564,4.0,80,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
5,Sonic Drive-In,TN,36.269593,-87.058943,2.0,6,"Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
9,Sonic Drive-In,TN,36.208102,-86.768170,1.5,10,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,..."
11,Vietnamese Food Truck,FL,27.955269,-82.456320,4.0,10,"Vietnamese, Food, Restaurants, Food Trucks"
12,Denny's,IN,39.637133,-86.127217,2.5,28,"American (Traditional), Restaurants, Diners, B..."
15,Tuna Bar,PA,39.953949,-75.143226,4.0,245,"Sushi Bars, Restaurants, Japanese"
19,BAP,PA,39.943223,-75.162568,4.5,205,"Korean, Restaurants"
22,Hibachi Express,IN,39.904320,-86.053080,4.0,20,"Steakhouses, Asian Fusion, Restaurants"
23,Romano's Macaroni Grill,NV,39.476117,-119.789339,2.5,339,"Restaurants, Italian"
29,Domino's Pizza,TN,36.464747,-86.659187,3.5,8,"Pizza, Chicken Wings, Sandwiches, Restaurants"


In [18]:
review_jsonO_path = './Obesity_2021.json'
data = pd.read_json(review_jsonO_path)

mapData = pd.merge(data, business_R, on='state', how='inner')

#merge latitude&longtitue into coordinates
mapData['coordinates'] = "[" + mapData['latitude'].astype(str) +","+ mapData["longitude"].astype(str) + "]"
mapData.groupby([mapData.latitude.round(10), mapData.longitude.round(10)]).ngroup()


key = []
count = 0
for i in mapData[mapData.columns[0]]:
    key.append(count)
    count = count + 1


mapData.insert(len(mapData.columns), 'key', key)
#drop redundent column
mapData = mapData.drop(['state', 'prevalence','categories','latitude', 'longitude'], axis=1)

#rename column
mapData = mapData.rename(columns={'state_Full': 'state', 'stars':'mean_rating', 'review_count': 'count'})

csv_name = "./mapData.csv"
mapData.to_csv(csv_name, index=False)
          
csvFilePath = r'./mapData.csv'
jsonFilePath = r'./mapData.json'
csv_to_json(csvFilePath, jsonFilePath)


In [19]:
####scatterPlot.json
scatterPlot = pd.merge(data, business_R, on='state', how='inner')

scatterPlot = scatterPlot.drop(['state', 'latitude', 'longitude', 'stars', 'review_count'], axis=1)
resFoodCount = []
fastFoodCount = []
for i in range(54):
    rescount = 0
    ffcount = 0
    for j in range(29900):
        if(scatterPlot.state_Full[j] == data.state_Full[i]):
            if(scatterPlot.categories.str.contains('Restaurants')[j]):
                rescount = rescount + 1
            if(scatterPlot.categories.str.contains('Fast Food')[j]):
                ffcount = ffcount + 1
                
    resFoodCount.append(rescount)
    fastFoodCount.append(ffcount)
        
print(resFoodCount)    


[0, 0, 1609, 0, 630, 0, 0, 601, 0, 5440, 0, 0, 0, 864, 632, 2545, 0, 0, 0, 2116, 0, 0, 0, 0, 0, 0, 2508, 0, 0, 972, 0, 2105, 0, 0, 0, 0, 0, 0, 0, 7163, 0, 0, 0, 0, 2715, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [20]:
review_jsonO_path = './Obesity_2021.json'
obesity = pd.read_json(review_jsonO_path)

obesity.insert(len(obesity.columns), 'resFoodCount', resFoodCount)
obesity.insert(len(obesity.columns), 'fastFoodCount', fastFoodCount)


scatterPlot = pd.merge(obesity, business_R, on='state', how='inner')
scatterPlot = scatterPlot.drop(['name', 'state', 'latitude', 'longitude', 'stars', 'review_count', 'categories'], axis=1)
scatterPlot = scatterPlot.drop_duplicates('state_Full')
scatterPlot = scatterPlot.rename(columns={'state_Full': 'state'})

key = []
count = 0
for i in scatterPlot[scatterPlot.columns[0]]:
    key.append(count)
    count = count + 1
scatterPlot.insert(len(scatterPlot.columns), 'key', key)

print(scatterPlot)

csv_name = "./scatterPlot.csv"
scatterPlot.to_csv(csv_name, index=False)

          
csvFilePath = r'./scatterPlot.csv'
jsonFilePath = r'./scatterPlot.json'
csv_to_json(csvFilePath, jsonFilePath)


              state prevalence  resFoodCount  fastFoodCount  key
0           Arizona       31.3          1609            440    0
1609     California       27.6           630             57    1
2239       Delaware       33.9           601             71    2
2840        Florida    No data          5440            889    3
8280          Idaho       31.6           864            185    4
9144       Illinois       34.2           632            198    5
9776        Indiana       36.3          2545            656    6
12321     Louisiana       38.6          2116            294    7
14437      Missouri       37.3          2508            565    8
16945        Nevada       31.3           972            187    9
17917    New Jersey       28.2          2105            251   10
20022  Pennsylvania       33.3          7163            655   11
27185     Tennessee       35.0          2715            555   12


In [28]:
###wordCloud.json
review_jsonO_path = './Obesity_2021.json'
data = pd.read_json(review_jsonO_path)

wordCloud = pd.merge(data, business_R, on='state', how='inner')

wordCloud = wordCloud.drop(['state', 'prevalence', 'latitude', 'longitude', 'stars', 'review_count'], axis=1)

cat = []
name = []

for name1 in wordCloud.name:
    name1 = name1.replace("'", r"=")
    name.append(name1)
    
for categories in wordCloud.categories:
    categories = categories.split(",")
    cat.append(categories)
    
    
    
wordCloud = wordCloud.drop(['categories'], axis=1)
wordCloud.insert(len(wordCloud.columns), 'categories', cat)
   

key = []
count = 0
for i in wordCloud[wordCloud.columns[0]]:
    key.append(count)
    count = count + 1
wordCloud.insert(len(wordCloud.columns), 'key', key)

wordCloud = wordCloud.rename(columns={'state_Full': 'state'})
#wordCloud.categories.infer_objects()

#SAVE wordCloud
csv_name = "./wordCloudData.csv"
wordCloud.to_csv(csv_name, index=False)


          
csvFilePath = r'./wordCloudData.csv'
jsonFilePath = r'./wordCloudData.json'
csv_to_json(csvFilePath, jsonFilePath)